In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import uniform
from scipy.optimize import minimize

In [10]:
# 定义模型参数
class ModelParams:
    def __init__(self, k, beta):
        self.k = k
        self.beta = beta
        
def get_centers(k):
    """根据 k 值确定两个类别的中心点"""
    centers = {
        1: ([0.25, 0.5, 0.5, 0.5], [0.75, 0.5, 0.5, 0.5]),  # x1 = 0.5 分割
        2: ([0.5, 0.25, 0.5, 0.5], [0.5, 0.75, 0.5, 0.5]),  # x2 = 0.5 分割
        3: ([0.5, 0.5, 0.25, 0.5], [0.5, 0.5, 0.75, 0.5]),  # x3 = 0.5 分割
        4: ([0.5, 0.5, 0.5, 0.25], [0.5, 0.5, 0.5, 0.75])   # x4 = 0.5 分割
    }
    return centers[int(k)]

# 定义似然函数
def likelihood(params, x, c, r):
    k, beta = int(params.k), params.beta
    
    # 获取对应的类别中心点
    center1, center2 = get_centers(k)
    
    # 计算到两个类别中心的距离
    d1 = np.linalg.norm(x - center1)
    d2 = np.linalg.norm(x - center2)
    
    # 计算选择概率
    p_c1 = np.exp(-beta * d1) / (np.exp(-beta * d1) + np.exp(-beta * d2))
    p_c2 = 1 - p_c1
    
    # 计算似然
    if c == 1:
        p_c = p_c1
    else:
        p_c = p_c2
    
    if r == 1:
        return p_c
    else:
        return 1 - p_c

# 定义对数似然函数
def log_likelihood(params, data):
    ll = 0
    for _, row in data.iterrows():
        x = row[['feature1', 'feature2', 'feature3', 'feature4']].values
        c = row['choice']
        r = row['feedback']
        ll += np.log(likelihood(params, x, c, r))
    return -ll  # 最小化负对数似然

# 定义先验分布
def prior(params):
    return uniform.pdf(params.k, 1, 4) * uniform.pdf(params.beta, 0, 10)

# 定义后验分布
def posterior(params, data):
    return log_likelihood(params, data) - np.log(prior(params))

# 修改优化函数以确保 k 是整数
def fit_model(data):
    def objective(params):
        k, beta = params
        return posterior(ModelParams(k=int(k), beta=beta), data)
    
    initial_params = [1, 1.0]  # k 初始值为 2，beta 初始值为 1.0
    bounds = [(1, 4), (0, 10)]  # k 的范围是 [1, 4]，beta 的范围是 [0, 10]
    
    result = minimize(
        objective,
        initial_params,
        method='L-BFGS-B',
        bounds=bounds
    )
    
    return ModelParams(k=int(round(result.x[0])), beta=result.x[1])

In [11]:
# 读取CSV文件
data = pd.read_csv('Task2.csv')

# 对每个被试拟合模型
for isub in data['iSub'].unique():
    sub_data = data[data['iSub'] == isub]
    fitted_params = fit_model(sub_data)
    print(f"Subject {isub}: k = {fitted_params.k:.2f}, beta = {fitted_params.beta:.2f}")

Subject 1: k = 1.00, beta = 10.00
Subject 4: k = 1.00, beta = 10.00
Subject 6: k = 1.00, beta = 3.30
Subject 11: k = 1.00, beta = 2.76
Subject 21: k = 1.00, beta = 3.04
Subject 26: k = 1.00, beta = 3.13
Subject 27: k = 1.00, beta = 2.06
